# What's the flow rate through JdF in the months we were comparing to CIOPS?

In [28]:
import xarray as xr
import numpy as np
import datetime
import matplotlib.pyplot as plt
import math
from pathlib import Path
from lo_tools import plotting_functions as lo_plot
from datetime import timedelta

In [2]:
grid = xr.open_dataset('/data1/parker/LO_data/grids/cas6/grid.nc')

x = 316
ymin = 897   
ymax = 946

In [3]:
def get_his_fn_from_dt(dt):

    path = Path("/data1/parker/LO_roms")
    # This creates the Path of a history file from its datetime
    if dt.hour == 0:
        # perfect restart does not write the 0001 file
        dt = dt - timedelta(days=1)
        his_num = '0025'
    else:
        his_num = ('0000' + str(dt.hour + 1))[-4:]
    date_string = dt.strftime('%Y.%m.%d')
    fn = path / 'cas6_v0_live' / ('f' + date_string) / ('ocean_his_' + his_num + '.nc')
    return fn

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c * 1000 # Distance in meters

    return distance

In [4]:
days = 30
# for the 2018 winter run
start = datetime.datetime(2018,2,7)
dates = [start + timedelta(hours=i) for i in range(days*24)]
fn = [get_his_fn_from_dt(date) for date in dates]

In [30]:
# since width of the JdF will remain constant throughout the run...
data = xr.open_dataset(fn[0])

ys = np.arange(ymin,ymax+2)

dist = [haversine(data.lat_u[ys[i],x], data.lon_u[ys[i],x], data.lat_u[ys[i+1],x], data.lon_u[ys[i+1],x]) for i in range(len(ys)-1)]


In [31]:
width, _ = np.meshgrid(dist, np.zeros(30))

In [16]:
# what if i calculated z differently and width differently..?
flow = 0

for f in fn:

    # dist = lo_plot.get_sect(f,'u',np.repeat(x,(ymax+1)-(ymin-1)),np.arange(ymin-1,ymax+1))[6]
    # # z = lo_plot.get_zfull(0,f,'u')[:,ymin:ymax+1,x]

    # width = dist[:,1:]-dist[:,:-1]

    data = xr.open_dataset(f)
    u = data.u.sel(eta_u=slice(ymin, ymax+1), xi_u=x).squeeze(dim='ocean_time')

    z = ((-1*data.Cs_w[::-1])*(grid.h+data.zeta[0]))[:,ymin:ymax+1,x]
    height = (z[1:,:].values-z[:-1,:].values)   

    flow += np.nansum(np.where(u>0, u, 0)*height*width)

In [17]:
flow/(len(dates))

431722.4379695686

In [26]:
# what if i calculated z differently (but didn't flip bc i'm not 110% sure that's correct) and width differently..?
# i should say that my hunch is that it is right to flip.. because i assume the the smallest spacing should be at the surface

flow = 0

for f in fn:

    # dist = lo_plot.get_sect(f,'u',np.repeat(x,(ymax+1)-(ymin-1)),np.arange(ymin-1,ymax+1))[6]
    # # z = lo_plot.get_zfull(0,f,'u')[:,ymin:ymax+1,x]

    # width = dist[:,1:]-dist[:,:-1]

    data = xr.open_dataset(f)
    u = data.u.sel(eta_u=slice(ymin, ymax+1), xi_u=x).squeeze(dim='ocean_time')

    z = (((1+data.Cs_w))*(grid.h+data.zeta[0]))[:,ymin:ymax+1,x]
    height = (z[1:,:].values-z[:-1,:].values)   

    flow += np.nansum(np.where(u>0, u, 0)*height*width)

In [27]:
flow/(len(dates))

584959.1971882147